# AI for BI

Generating a chart

We'll use CrewAI for the processing. This includes a Python interpreter which will be used to create the charts. You must have Docker installed locally to run the code automatically.

Running code generated by an LLM is potentially dangerous as there is n guarantee that the code wil be safe. Running the code in a Docker container, however, provides a sandbox for the code to run in and isolates it from the application code and local operating system.


Import the necessary libraries and set the LLM model (defaults to using OpenAI)

In [1]:
from crewai import Agent, Task, Crew

llm = "gpt-4o-mini"

Import the tools required to read and write files

In [2]:
from crewai_tools import tool
from crewai_tools import FileReadTool, FileWriterTool

# Initialize the tool to read any files the agents knows or lean the path for
file_read_tool = FileReadTool()
file_writer_tool = FileWriterTool()

Set up the ``chart_agent``

In [3]:
# Define agent

chart_agent = Agent(
        role="Chart creator",
        goal="""Read the data provided and create a matplotlibb chart from that data.
                If you are given specific instructions on how to draw the chart then follow them,
                if not then create a chart that best represents the data""",
        backstory="You aim is to read and analyse sales data and create a mathplotlib chart",
        tools=[file_read_tool, file_writer_tool],
        allow_delegation=False,
        llm=llm,
        allow_code_execution=True
    )



The following defines a function that will create a task to create a markdown summary and chart for each data file and set up a crew.

We define the files and then loop through them calling the ``create_charts`` function to write the individual markdown files.


In [4]:
files = {
        'data_file_name':'./sales_product_cat.csv',
        'chart_file_name': './sales_product_summary.png',
    }

In [ ]:

create_chart = Task(
    description=f"Create a chart for {files['data_file_name']} and save it in {files['chart_file_name']}",
    expected_output="""A matplotlib chart""",
    agent=chart_agent,
    tools=[file_read_tool, file_writer_tool]
)

# Define the crew
crew = Crew(
    agents=[chart_agent],
    tasks=[create_chart],
    verbose=True
)
result = crew.kickoff()
#task_output = create_chart.output

# Agent: Chart creator
## Task: Create a chart for ./sales_product_cat.csv and save it in ./sales_product_summary.png


# Agent: Chart creator
## Thought: I need to read the sales data from the provided CSV file to create a chart.
## Using tool: Read a file's content
## Tool Input: 
"{\"file_path\": \"./sales_product_cat.csv\"}"
## Tool Output: 
Product Category,Product Name,Units Sold,Revenue ($),% Change (Month-over-Month)
Electronics,Smart TV,1500,450000,+10%
Electronics,Laptop,1200,840000,+5%
Electronics,Smartphone,2000,1200000,-2%
Home Appliances,Refrigerator,700,350000,+8%
Home Appliances,Washing Machine,500,250000,-3%
Home Appliances,Microwave,900,135000,+15%
Furniture,Sofa,300,90000,+20%
Furniture,Dining Table,150,75000,+12%
Furniture,Bed Frame,250,125000,+7%



# Agent: Chart creator
## Thought: Thought: I have retrieved the sales data. Now, I need to analyze it to create a suitable chart representing the sales performance across different product categories.
## Using tool: Co

In [6]:
#print(f"Task Description: {result.description}")
#print(f"Task Summary: {result.summary}")
print(f"Raw Output: {result.raw}")

Raw Output: I have created the matplotlib chart representing the total revenue by product category and saved it to `./sales_product_summary.png`.


In [8]:
create_chart = Task(
    description=f"""Create a chart for {files['data_file_name']} and save it in 'detailed_chart.png'.'
                    The chart should be a vertical bar chart where the x-axis is the 'Product Name',
                    the y-axis is 'Revenue ($), and the bars are colured by 'Product Category'""",
    expected_output="""A matplotlib chart""",
    agent=chart_agent,
    tools=[file_read_tool, file_writer_tool]
)

# Define the crew
crew = Crew(
    agents=[chart_agent],
    tasks=[create_chart],
    verbose=True
)
result = crew.kickoff()

2024-11-09 17:10:53,607 - 9644 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Chart creator
## Task: Create a chart for ./sales_product_cat.csv and save it in 'detailed_chart.png'.'
                    The chart should be a vertical bar chart where the x-axis is the 'Product Name',
                    the y-axis is 'Revenue ($), and the bars are colured by 'Product Category'


# Agent: Chart creator
## Thought: I need to read the content of the "./sales_product_cat.csv" file to analyze the sales data, which will allow me to create a vertical bar chart as specified.
## Using tool: Read a file's content
## Tool Input: 
"{\"file_path\": \"./sales_product_cat.csv\"}"
## Tool Output: 
Product Category,Product Name,Units Sold,Revenue ($),% Change (Month-over-Month)
Electronics,Smart TV,1500,450000,+10%
Electronics,Laptop,1200,840000,+5%
Electronics,Smartphone,2000,1200000,-2%
Home Appliances,Refrigerator,700,350000,+8%
Home Appliances,Washing Machine,500,250000,-3%
Home Appliances,Microwave,900,135000,+15%
Furniture,Sofa,300,90000,+20%
Furniture,Dining Table,

![](detailed_chart.png)

In [ ]:
#%pip install plotly
#%pip install kaleido==0.1.0.post1
import plotly.express as px
import pandas as pd

df = pd.read_csv("sales_product_cat.csv")
fig = px.bar(df, x="Product Name", y="Revenue ($)", 
             color="Product Category", template="plotly_white")
#fig.show()
fig.write_image("plotly_image.png")


![](image.png)